In [1]:
#原來一般的WebDriver
from selenium import webdriver
from selenium.webdriver import chrome
from selenium.webdriver import firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import time
from queue import Queue
import codecs
from threading import Thread
import threading
from urllib.request import urlopen
import json
import os
import random
import csv
WINDOW_SIZE = "1000,1000"   #窗体大小
EDGEDRIVER_PATH=r"C:/Users/HP/microsoft_webdriver/MicrosoftWebDriver.exe"   # edge_driver 路徑
FIREFOXDRIVER_PATH=r"C:/Users/HP/geckodriver/geckodriver.exe"    # firefox_driver 路徑
FIREFOX_PATH=r"C:/Program Files/Mozilla Firefox/firefox.exe"     # firefox 路徑
firefox_options = firefox.options.Options()
firefox_options.add_argument("--headless")
firefox_options.add_argument("--window-size=%s" % WINDOW_SIZE)
firefox_options.binary_location = FIREFOX_PATH
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("dom.webnotifications.enabled", False)    # firefox disable notification

CHROME_PATH = r"C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"  # chrome 路径
CHROMEDRIVER_PATH = r"C:/Users/HP/chromedriver_win32/chromedriver.exe" # chrome_driver 路径  
chrome_options = chrome.options.Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito") #無痕模式
#禁止圖片加載跟跳通知
prefs = {"profile.default_content_setting_values.notifications" : 2, "profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
chrome_options.binary_location = CHROME_PATH

In [17]:
#另外一個有關聯人的所有id，看會不會出現在family中
#目前篩選的規則是只要有符合任何一項我下的條件就把他的url記錄下來
#nominal的大概處理完
#人的部分從family開始 還有塗鴉牆上所有按讚的
# a,b,c,d 為分別查找的name列表
#篩選條件

def namelist(filename):
    name=codecs.open(filename+"namelist.txt","r+","utf-8")
    a=name.read().split("\n")
    del a[len(a)-1]
    name.close()
    return a

#將namelist中的姓名、url、id全部合併在一起
def total(names):
    total=[]
    for i in names:
        total.extend(namelist(i))
    return total

def comn(names):
    comn=[]        #收集所有的id
    t=total(names)
    for w in range(len(t)):
        if w%3==2:
            comn.append(t[w])
    return comn

def identifier(filename, ocr_addr, ocr_brdy_phone, work_comp_job_name, comn):
    info=[]
    all_info={}
    with open(filename+"_data.json","r+") as file:
        for line in file.readlines():
            dic = json.loads(line)
            for key, value in dic.items():
                all_info[key]=value 
    for i in all_info:
        if ocr_addr is not []:
            for j in all_info[i]["addr"]:
                for k in ocr_addr:
                    if j.find(k) !=-1:
                        if i not in info:
                            info.append(i)
            for m in all_info[i]["edu_work"]:
                for k in ocr_addr:
                    if m.find(k)!=-1:
                        if i not in info:
                            info.append(i)
        if work_comp_job_name is not []:
            for m in all_info[i]["edu_work"]:
                for s in work_comp_job_name:
                    if m.find(s)!=-1:
                        if i not in info:
                            info.append(i)
        if ocr_brdy_phone is not []:
            for k in all_info[i]["contact"]:
                for l in ocr_brdy_phone:
                    if k.find(l) !=-1:
                        if i not in info:
                            info.append(i)
        #如果在某個人的家人列表中有找到其他同名的人的話收集起來
        if comn is not []:
            for n in all_info[i]["family"].values():    #values()會回傳字典中所以的值  以這個part的話是id
                for w in comn: #comn中的是一堆id 
                    if n.find(w) !=-1:
                        if i not in info:
                            info.append(i)
    return info

In [9]:
#判斷捲完了沒有
def finish_roll_friend(driver):
    try:
        t=driver.find_element_by_xpath('//*[@id="timeline-medley"]/div/div[2]/div[1]/div/div/h3')
    except NoSuchElementException:
        t=0
    return t
#每個同名的人的好友列表，期望是只有一個URL可以找，表示已經確定為誰
#目標是查下來列表後，看看族群中其他人有沒有在這個朋友列表中
#單人
def friends_list(url, keyword, email0, password0, email1, password1, email2, password2, email3, password3):
    rn=random.randrange(2,5)
    friend={}
    data=codecs.open(keyword+"_friend_list.json", "w+")
    for i in range(0, len(url), rn):
        d=i/rn
        if d%4==0: 
            browser="chrome"
            driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
            driver.get("https://www.facebook.com/")
            time.sleep(5)
            user=driver.find_element_by_css_selector("#email")
            user.send_keys(email0)
            time.sleep(1)
            pwd=driver.find_element_by_css_selector("#pass")
            pwd.send_keys(password0)
            time.sleep(1)
            log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
            log.submit()
            time.sleep(5)
            
        elif d%4==1:
            try:
                browser="firefox"
                driver=webdriver.Firefox(executable_path=FIREFOXDRIVER_PATH, firefox_options=firefox_options, firefox_profile=firefox_profile)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email1)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password1)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                
            except FileNotFoundError:
                browser="chrome"
                driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email1)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password1)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
               
        elif d%4==2:
            browser="chrome"
            driver=webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
            driver.get("https://www.facebook.com/")
            time.sleep(5)
            user=driver.find_element_by_css_selector("#email")
            user.send_keys(email2)
            time.sleep(1)
            pwd=driver.find_element_by_css_selector("#pass")
            pwd.send_keys(password2)
            time.sleep(1)
            log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
            log.submit()
            time.sleep(5)
            
        elif d%4==3:
            try:
                browser="firefox"
                driver=webdriver.Firefox(executable_path=FIREFOXDRIVER_PATH, firefox_options=firefox_options, firefox_profile=firefox_profile)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email3)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password3)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                
            except FileNotFoundError:
                browser="chrome"
                driver= webdriver.Chrome(executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options)
                driver.get("https://www.facebook.com/")
                time.sleep(5)
                user=driver.find_element_by_css_selector("#email")
                user.send_keys(email3)
                time.sleep(1)
                pwd=driver.find_element_by_css_selector("#pass")
                pwd.send_keys(password3)
                time.sleep(1)
                log=driver.find_element_by_xpath('//label[@class="uiButton uiButtonConfirm"]/input')
                log.submit()
                time.sleep(5)
                
        for n in range(i, i+rn):
            if n<len(url):
                driver.get("https://www.facebook.com/profile.php?id="+url[n])
                time.sleep(2)
            else:
                break
            for _ in range(15):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
            wow=driver.find_elements_by_css_selector('[class="_2x4v"]')
            great={}
            if len(wow)==0:
                print("-----------po文都沒人幫他點讚~~~哭哭------------")
            else:
                w=[]
                for i in wow:      
                    w.append(i.get_attribute("href"))
                for i in w:
                    driver.get(i)
                    time.sleep(2)
                    ha=driver.find_elements_by_xpath('//div[@class="_5j0e fsl fwb fcb"]/a')
                    for j in ha:
                        if eval(j.get_attribute("data-gt"))["engagement"]["eng_tid"] not in great:
                            great[eval(j.get_attribute("data-gt"))["engagement"]["eng_tid"]]=j.text #抓的是id
            driver.get("https://www.facebook.com/profile.php?id="+url[n])
            time.sleep(3)
            driver.get(driver.find_elements_by_css_selector('[class="_6-6"]')[1].get_attribute("href"))
            time.sleep(3)
            fd=driver.find_elements_by_xpath('//div[@class="fsl fwb fcb"]/a')
            friends={}
            if len(fd)==0:
                print("-----------他的朋友不給看-----------")
            else: 
                count=0
                while finish_roll_friend(driver)==0:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    print(count)
                    count=count+1
                    time.sleep(1.5)
                frnd=driver.find_elements_by_xpath('//div[@class="fsl fwb fcb"]/a')
                for i in frnd:
                    if eval(i.get_attribute("data-gt"))["engagement"]["eng_tid"] not in friends:
                        friends[eval(i.get_attribute("data-gt"))["engagement"]["eng_tid"]]=i.text     #id:人名
            friends.update(great)          #看一個人按讚的用戶、朋友列表的聯集   dict的update本身就有取聯集的效果`,以ID為INDEX
            friend[url[n]]=friends        #收集起來變dict，以當個人當index，值也是dict
    json.dump(friend, data)
    data.close()
    return friend #dict id當作index

In [10]:
#由value回去找key，回傳list
def get_key(dic, value):
    return [k for k, v in dic.items() if v == value]
#要看朋友列表中有沒有其他人的id
def friend_identify(keyword,friend, target):#keyword要是人名，friend要是dict with index是id
    goal=[]
    for i in friend:
        for j in friend[i].values():  #j中是各種人的人名
            for k in target:
                if k!=keyword:
                    if j.find(k)!=-1:     #代表有找到那個人名
                        goal.append(i)     #存id
                        goal.extend(get_key(friend[i],k))
    goal=list(set(goal))
    return goal

#接著再做確認名單的好友合併  星期五的part
def total_f(url,keyword, target, email0,password0, email1, password1,email2, password2,email3, password3):
    friend=friends_list(url, keyword, email0,password0, email1, password1,email2, password2,email3, password3)
    return friend_identify(keyword,friend, target)

In [ ]:
#找取有關聯的人--交集
def connections(target, goal):
    all_info={}
    for keyword in target:
        file=codecs.open(keyword+"_friend_list.json", "r+")
        for line in file.readlines():
            dic = json.loads(line)
            for key, value in dic.items():
                all_info[key]=value
    file.close()
    i=0
    p=set(all_info[goal[i]].keys())
    while i+1<len(goal):
        r = p & set(all_info[goal[i+1]].keys())
        p=r
        i=i+1
    p=list(p)
    return p  #交集後的list

In [ ]:
target=["吳俊霖","吳俊樺","黃秋敏","黃秋米","吳俊融"]

#想要用的瀏覽器及輪換的帳號(目前設定四組)
browser="Chrome"
email0="123456@gmail.com"
password0="123"
email1="123456@yahoo.com.tw"
password1="123"
email2="123456789@yahoo.com.tw"
password2="123"
email3="123@gmail.com"
password3="123"

goal=[]
#吳俊霖--["100003188190469"]
url=identifier(filename, ocr_addr, ocr_brdy_phone, work_comp_job_name, comn(target))
goal.extend(total_f(url, "吳俊霖", target,email0,password0, email1, password1,email2, password2,email3, password3))
#吳俊融--["100004985381617"]
url=identifier(filename, ocr_addr, ocr_brdy_phone, work_comp_job_name, comn(target))
goal.extend(total_f(url, "吳俊融", target,email0,password0, email1, password1,email2, password2,email3, password3))
#吳俊樺
url=identifier("吳俊樺", ["花蓮", "吉安"], ["1981年1月23日", "1月23日", "0912345678"], ["開車","開救護車"], comn(target)) 
goal.extend(total_f(url, "吳俊樺", target,email0,password0, email1, password1,email2, password2,email3, password3))
#黃秋敏
url=identifier("黃秋敏", ["花蓮", "吉安"], ["1960年1月23日", "1月23日", "0912345678"],["家管"], comn(target)) 
goal.extend(total_f(url, "黃秋敏", target, email0, password0, email1, password1,email2, password2,email3, password3))
#黃秋米
url=identifier("黃秋米", ["花蓮", "吉安"], ["1960年1月23日", "1月23日", "0912345678"],["家管"], comn(target)) 
goal.extend(total_f(url, "黃秋米", target, email0, password0, email1, password1,email2, password2,email3, password3))
goal=connections(target, goal)